In [1]:
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
from collections import Counter
from yellowbrick.text import PosTagVisualizer
import contractions
import os

In [13]:
#load the csv into a pandas dataframe
#please change the directory according to where you stored the file 
df = pd.read_csv("tweets_dataset.csv", encoding = "latin-1")
df.columns = ["class", "Index", "Date", "No_Query", "Username", "text"]
df

,class,Index,Date,No_Query,Username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
df1= pd.read_csv("Suicide_Detection.csv", encoding = "latin-1")
df1.columns = ["Index", "text", "class"]
df1.drop("Index", inplace = True, axis = 1)
df1

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"Iâm so lostHello, my name is Adam (16) and I...",suicide
...,...,...
232069,If you don't like rock then your not going to ...,non-suicide
232070,You how you can tell i have so many friends an...,non-suicide
232071,pee probably tastes like salty teaðð¦â¼ï...,non-suicide
232072,The usual stuff you find hereI'm not posting t...,suicide


In [4]:
#remove the column with no siginificance 
df.columns = ["class", "Index", "Date", "No_Query", "Username", "text"]
df.drop("No_Query", inplace = True, axis = 1)

In [5]:
#seperate the columns into training labels and features 
df_text = df["text"]
df_class = df["class"]

In [6]:
df1_text = df1["text"]
df1_class = df1["class"]

In [7]:
#concat the label and features columns into a new dataframe
df_concat = pd.concat([df_text, df_class], axis = 1)
df_concat.columns = ["text","class"]
df_concat

,text,class
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0
...,...,...
1599994,Just woke up. Having no school is the best fee...,4
1599995,TheWDB.com - Very cool to hear old Walt interv...,4
1599996,Are you ready for your MoJo Makeover? Ask me f...,4
1599997,Happy 38th Birthday to my boo of alll time!!! ...,4


In [8]:
splits = list(df1.groupby("class"))
non_suicide = splits[0][1]
suicide = splits[1][1]
df1_concat = pd.concat([suicide, non_suicide], axis = 0)
df1_concat

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
3,i need helpjust help me im crying so hard,suicide
4,"Iâm so lostHello, my name is Adam (16) and I...",suicide
5,Honetly idkI dont know what im even doing here...,suicide
6,[Trigger warning] Excuse for self inflicted bu...,suicide
...,...,...
232068,I did something today I went sledding with my ...,non-suicide
232069,If you don't like rock then your not going to ...,non-suicide
232070,You how you can tell i have so many friends an...,non-suicide
232071,pee probably tastes like salty teaðð¦â¼ï...,non-suicide


In [9]:
new_df_concat = pd.concat([df_concat, df1_concat], axis = 0)
new_df_concat

,text,class
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0
...,...,...
232068,I did something today I went sledding with my ...,non-suicide
232069,If you don't like rock then your not going to ...,non-suicide
232070,You how you can tell i have so many friends an...,non-suicide
232071,pee probably tastes like salty teaðð¦â¼ï...,non-suicide


In [10]:
#removing usernames that starts with @ because usernames are of little significance in detecting depression in the texts
new_df_concat["text"] = new_df_concat["text"].str.replace("@[^\s]+", "", regex = True)



KeyboardInterrupt: 

In [ ]:
#removing punctuation 
new_df_concat["text"] = new_df_concat["text"].str.replace(r'[^\w\s]', '', regex = True)

In [ ]:
#encode the characters into ascii and ignore it if the character can't be encoded to ascii, therefore, special characters 
#such as emoticons, Chinese characters, greek characters etc. that are not in the ascii are removed during the encoding. 
#Then, decode the texts again to latin-1

new_df_concat["text"] = new_df_concat["text"].apply(lambda x : x.encode("ascii", "ignore").decode())

In [ ]:
#extending the contractions, eg. I'm to I am 
new_df_concat['text'] = new_df_concat['text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
new_df_concat["text"] = [' '.join(map(str,l)) for l in new_df_concat["text"]]


In [ ]:
new_df_concat

In [ ]:
#from nltk.corpus import stopwords 
#stop_words = set(stopwords.words('english'))

#stopwords
#df_concat["Tweet"] = df_concat["Tweet"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)])) 

In [ ]:
#tokenize the words
from nltk.tokenize import word_tokenize
new_df_concat["text_tokenize"] = new_df_concat["text"].apply(word_tokenize)


In [ ]:
new_df_concat["joined_text"] = [' '.join(map(str,l)) for l in new_df_concat["text_tokenize"]]
new_df_concat.head() 

In [ ]:
new_df_concat['text'].replace('', np.nan, inplace=True)
new_df_concat.dropna(subset=['text'], inplace=True)

In [ ]:
new_df_concat

In [126]:
#save the data to csv 
new_df_concat.to_csv("L:\\ML-Assignment\\training_data.csv", index = False, columns = ["joined_text", "class"])